In [132]:
import pandas as pd
import dash
from dash import dcc, html

In [255]:
df=pd.read_csv(r'/Users/katherinewu/Documents/takehome/course_completion_data.csv')
df2=pd.read_csv(r'/Users/katherinewu/Documents/takehome/feedback_data.csv')
df3=pd.read_csv(r'/Users/katherinewu/Documents/takehome/student_engagement_data.csv')


course_completion=pd.DataFrame(df)
feedback_data=pd.DataFrame(df2)
student_engagement_data=pd.DataFrame(df3)




In [258]:
feedback_data
student_engagement_data
#根据学生ID和课程ID在student engagement表里查找会话时间，返回结果为0
df4=pd.merge(feedback_data,student_engagement_data,on=['StudentID','LessonID'])
df4

,StudentID,LessonID,FeedbackScore,FeedbackComment,SessionTime


In [ ]:
#学生是否有频繁的会话变化
#按照学生号课程号进行计数
student_engagement_data_count=student_engagement_data.groupby(['StudentID','LessonID']).size()
#找出大于10的学生号和课程号(假定同一个学生登陆同一个课程频次超过10次为异常值)
student_engagement_data_count2 = student_engagement_data_count[student_engagement_data_count>10].index.tolist()
#删除异常值
cleaned_student_engagement_data = student_engagement_data[~(student_engagement_data['StudentID'].isin([c for c, _ in student_engagement_data_count2]) 
                                        & student_engagement_data['LessonID'].isin([i for _, i in student_engagement_data_count2]))]
print(cleaned_student_engagement_data)


In [253]:
#是否有某个学生会话时间明显偏离大部分学生

# 计算每门课程的均值和标准差
student_engagement_data_mean = student_engagement_data.groupby('LessonID')['SessionTime'].mean()
student_engagement_data_std = student_engagement_data.groupby('LessonID')['SessionTime'].std()

# 计算样本均值加减两倍标准差的数据
two_std_above = student_engagement_data_mean + 2 * student_engagement_data_std
two_std_below = student_engagement_data_mean - 2 * student_engagement_data_std

## 将两个 Series 与 DataFrame 进行对应
two_std_above_series = pd.Series(two_std_above.values, index=two_std_above.index)
two_std_below_series = pd.Series(two_std_below.values, index=two_std_below.index)
cleaned_student_engagement_data['two_std_above'] = cleaned_student_engagement_data['LessonID'].map(two_std_above_series)
cleaned_student_engagement_data['two_std_below'] = cleaned_student_engagement_data['LessonID'].map(two_std_below_series)

# 筛选数据
data_above_two_std = cleaned_student_engagement_data[cleaned_student_engagement_data['SessionTime'] > cleaned_student_engagement_data['two_std_above']]
data_below_two_std = cleaned_student_engagement_data[cleaned_student_engagement_data['SessionTime'] < cleaned_student_engagement_data['two_std_below']]

# 获取要删除的索引
indices_to_remove = pd.concat([data_above_two_std, data_below_two_std]).index
#indices_to_remove1 = pd.concat([data_above_two_std, data_below_two_std])

cleaned_student_engagement_data.drop(indices_to_remove, inplace=True)













In [231]:
#对于反馈评分表数据不一致进行清洗
feedback_data
feedback_data.loc[feedback_data['FeedbackComment'] == 'Poor', 'FeedbackScore'] = 1
feedback_data.loc[feedback_data['FeedbackComment'] == 'Needs Improvement', 'FeedbackScore'] = 2
feedback_data.loc[feedback_data['FeedbackComment'] == 'Average', 'FeedbackScore'] = 3
feedback_data.loc[feedback_data['FeedbackComment'] == 'Good', 'FeedbackScore'] = 4
feedback_data.loc[feedback_data['FeedbackComment'] == 'Excellent', 'FeedbackScore'] = 5
feedback_data

with pd.ExcelWriter('output.xlsx', mode='a') as writer:
    feedback_data.to_excel(writer, sheet_name='feedbackdata')






In [116]:
#参考代码（无效）
s_e_data_mean=pd.DataFrame(student_engagement_data_mean)
s_e_data_mean.reset_index(inplace=True)
s_e_data_mean=s_e_data_mean[['LessonID','SessionTime']]
#s_e_data_mean["SessionTime"]=s_e_data_mean['SessionTime']/60

merged_data=pd.merge(course_completion,s_e_data_mean,on='LessonID')
merged_data


cleaned_student_engagement_data_mean = cleaned_student_engagement_data.groupby('LessonID')['SessionTime'].mean()



,LessonID,ExpectedDuration,CompletionRate,SessionTime
0,1,5.000000,1.000000,31.846652
1,2,6.315789,0.868421,36.581036
2,3,7.631579,0.736842,41.160831
3,4,8.947368,0.605263,34.648384
4,5,10.263158,0.500000,38.083698
5,6,11.578947,0.500000,44.632972
6,7,12.894737,0.500000,29.166045
7,8,14.210526,0.500000,27.446975
8,9,15.526316,0.500000,27.770178
9,10,16.842105,0.500000,26.081173


In [254]:
compared_data=pd.merge(cleaned_student_engagement_data,course_completion,on=['LessonID'])

#会话时间与预期时间相比

#算出超过每个课程预期时长的学生参与数量和每个课程学生的总参与数量，并得出超出预期的参与比例
compared_data['minus'] = (compared_data['SessionTime']-compared_data['ExpectedDuration'])>0
proportion = compared_data.groupby('LessonID')['minus'].value_counts()
sum_data = compared_data.groupby('LessonID')['minus'].count()
rate_data=proportion/sum_data

rate_data=pd.DataFrame(rate_data)
rate_data.reset_index(inplace=True)

rate_data.rename(columns={0: 'Participationrate'}, inplace=True)

rate_data=rate_data[rate_data['minus'] == True]

rate_data=pd.merge(rate_data,course_completion[['LessonID','CompletionRate']],on='LessonID')

rate_data
#保存对比数据
with pd.ExcelWriter('output.xlsx', mode='a') as writer:
    rate_data.to_excel(writer, sheet_name='Expectedmanagement')







,LessonID,minus,Participationrate,CompletionRate
0,1,True,1.000000,1.000000
1,2,True,0.960000,0.868421
2,3,True,0.866667,0.736842
3,4,True,0.795455,0.605263
4,5,True,0.736842,0.500000
5,6,True,0.781818,0.500000
6,7,True,0.404762,0.500000
7,8,True,0.396226,0.500000
8,9,True,0.285714,0.500000
9,10,True,0.375000,0.500000


In [ ]:
#平均engagement time
engagement_data_mean = student_engagement_data['SessionTime'].mean()
engagement_data_mean
student_engagement_data_mean
student_engagement_data_avg=pd.DataFrame(student_engagement_data_mean)

student_engagement_data_avg.reset_index(inplace=True)
student_engagement_data_avg['average_engeagement_time']=engagement_data_mean
student_engagement_data_avg
student_engagement_data_avg.to_excel('student_engagement_data_avg.xlsx',sheet_name='student_engagement_data')

In [194]:
#最受欢迎的课程
compared_data=compared_data.sort_values(by='LessonID',ascending=False)

involoved_lesson=compared_data.groupby('LessonID')['StudentID'].size()
max_count = involoved_lesson.max()
max_categories = involoved_lesson[involoved_lesson == max_count].index.tolist()
most_popular_course=pd.DataFrame()
most_popular_course['LessonId']=max_categories
most_popular_course['StudentNumber']=max_count
most_popular_course
with pd.ExcelWriter('output.xlsx', mode='a') as writer:
    most_popular_course.to_excel(writer, sheet_name='MostPopular')

,LessonId,studentNumber
0,17,61
1,18,61
2,20,61


In [217]:
#参与学生总数
TotalStudents=student_engagement_data['StudentID'].drop_duplicates().count()
#平均完成度
avg_sessiontime=course_completion['CompletionRate'].mean()
Abstract_data= pd.DataFrame({'TotalStudents': [TotalStudents],'avg_sessiontime':[avg_sessiontime]})

Abstract_data
with pd.ExcelWriter('output.xlsx', mode='a') as writer:
    Abstract_data.to_excel(writer, sheet_name='Abstract_data')

,TotalStudents,avg_sessiontime
0,650,0.560526
